In [ ]:
%pip install -qqqU langchain-openai langgraph

In [ ]:
import os

from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_aws import BedrockEmbeddings

embeddings = BedrockEmbeddings()

qdrant_client = QdrantClient(
    location=os.getenv("VECTOR_STORE_URL", "http://localhost:6333"),
    api_key=os.getenv("VECTOR_STORE_API_KEY", "api_key"),
)

vector_store = QdrantVectorStore(
    client=qdrant_client, collection_name="regulamento_dos_cursos_de_graduacao_da_UFRN", embedding=embeddings
)

retriever = vector_store.as_retriever()

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "regulamentos_graducacao",
    "Artigos do Regulamento dos Cursos Regulares de Graduação"
    )

tools = [retriever_tool]

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=tools,
)

In [ ]:
inputs = {
    "messages": [
        ("user", "Como cancelar meu curso?")
    ]
}

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

print_stream(agent.stream(inputs, stream_mode="values"))